In [1]:
import skimage.io as io
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
sample_size=1200
n_components=150

In [2]:
label_data=pd.read_csv('labels.csv')
label_data.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [3]:
le=LabelEncoder()
label_data['breed_no']=le.fit_transform(label_data['breed'])

In [4]:
label_data.head()

,id,breed,breed_no
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,19
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,37
2,001cdf01b096e06d78e9e5112d419397,pekinese,85
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,15
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,49


In [5]:
label_data['filenames']=label_data['id'].map(lambda x:'train/'+x+'.jpg')

In [6]:
label_data.head()

,id,breed,breed_no,filenames
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,19,train/000bec180eb18c7604dcecc8fe0dba07.jpg
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,37,train/001513dfcb2ffafc82cccf4d8bbaba97.jpg
2,001cdf01b096e06d78e9e5112d419397,pekinese,85,train/001cdf01b096e06d78e9e5112d419397.jpg
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,15,train/00214f311d5d2247d5dfe4fe24b2303d.jpg
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,49,train/0021f9ceb3235effd7fcde7f7538ed62.jpg


In [7]:
image_data=io.imread_collection(label_data['filenames'])

In [8]:
labels=np.array(label_data.breed_no)

In [9]:
labels=labels[:sample_size]

In [10]:
resized_images=[]
for i in image_data[:sample_size]:
    resized_images.append(resize(i,(200,200)).ravel())

C:\Users\deban\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [11]:
training=np.array(resized_images)

In [12]:
training.shape

(1200, 120000)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training, labels,test_size=0.25, random_state=42)

In [14]:
from sklearn.decomposition import PCA

In [15]:
pca=PCA(n_components=n_components,whiten=True).fit(X_train)

In [16]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [17]:
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [19]:
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_pca, y_train)

C:\Users\deban\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [20]:
clf.predict(X_test_pca[:5])

array([119,   2, 109,  10,  70], dtype=int64)

In [21]:
X_train_pca.shape

(900, 150)

In [22]:
clf.score(X_test_pca,y_test)

0.013333333333333334

In [23]:
labels.shape

(1200,)

In [24]:
training.shape

(1200, 120000)